<a href="https://colab.research.google.com/github/daystram/ml-playground/blob/master/04_gym_acrobot_keras/04_gym_acrobot_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 Gym Acrobot using Keras

In [10]:
!rm -r ml-playground > /dev/null 2>&1
!git clone https://github.com/daystram/ml-playground.git
!sh ml-playground/helper/setup.sh > /dev/null 2>&1

Cloning into 'ml-playground'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 120 (delta 67), reused 64 (delta 27), pack-reused 0
Receiving objects: 100% (120/120), 95.41 KiB | 2.07 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [0]:
import sys
sys.path.append('/content/ml-playground')

In [12]:
import gym

env = gym.make("Acrobot-v1")
print(env.observation_space)
print(env.action_space)

Box(6,)
Discrete(3)


In [13]:
import numpy as np

acc_fitness = []
training_data = []

sample_count = 2000
steps = 750
goal_tres = 0.3

for i in range(sample_count):
  fitness = 0
  action_seq = []
  prev_observation = env.reset()
  time = 0
  
  for _ in range(steps):
    time += 1
    action = np.random.randint(-1, 2)
    action_seq.append([prev_observation, action])
    prev_observation, _, done, _ = env.step(action)
    if (-prev_observation[0] - prev_observation[2]) + 2 > 1:
      fitness += 1
    if done: break
  if fitness / time > goal_tres:
    acc_fitness.append(fitness / time)
    for seq in action_seq:
      if seq[1] == 1:
        action = [0, 0, 1]
      elif seq[1] == 0:
        action = [0, 1 ,0]
      else:
        action = [1, 0, 0]
      training_data.append([seq[0], action])
env.close()
print("{} samples".format(len(training_data)))
print(np.array(acc_fitness))

131259 samples
[0.302      0.33       0.394      0.47797357 0.3582996  0.342
 0.4        0.336      0.314      0.308      0.314      0.40942928
 0.406      0.41       0.348      0.372      0.454      0.368
 0.304      0.44094488 0.41625616 0.386      0.378      0.308
 0.408      0.328      0.35       0.366      0.318      0.43
 0.346      0.35294118 0.308      0.398      0.308      0.366
 0.314      0.362      0.322      0.36494845 0.308      0.406
 0.306      0.334      0.424      0.308      0.344      0.368
 0.302      0.332      0.454      0.352      0.344      0.41035857
 0.302      0.338      0.436      0.45       0.358      0.332
 0.37868481 0.33       0.308      0.33       0.35013263 0.432
 0.406      0.376      0.314      0.34       0.302      0.318
 0.492      0.388      0.38       0.318      0.314      0.362
 0.342      0.42325581 0.314      0.32       0.318      0.318
 0.36253041 0.32       0.36       0.33       0.308      0.322
 0.346      0.42       0.322      0.408      0

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

drop_rate = 0.3
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(6,)))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(512, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(3, activation='linear'))

model.compile(loss='mse',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])

model.fit(X, y, epochs=5, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
134560/134560 [==============================] - 10s 77us/step - loss: 0.2242 - acc: 0.3439
Epoch 2/5
134560/134560 [==============================] - 9s 68us/step - loss: 0.2224 - acc: 0.3451
Epoch 3/5
134560/134560 [==============================] - 9s 67us/step - loss: 0.2222 - acc: 0.3473
Epoch 4/5
134560/134560 [==============================] - 9s 68us/step - loss: 0.2220 - acc: 0.3485
Epoch 5/5
134560/134560 [==============================] - 9s 68us/step - loss: 0.2220 - acc: 0.3502


Epoch 1/5
131259/131259 [==============================] - 10s 73us/step - loss: 0.2243 - acc: 0.3426
Epoch 2/5
131259/131259 [==============================] - 9s 71us/step - loss: 0.2224 - acc: 0.3465
Epoch 3/5
131259/131259 [==============================] - 9s 69us/step - loss: 0.2222 - acc: 0.3458
Epoch 4/5
131259/131259 [==============================] - 9s 69us/step - loss: 0.2221 - acc: 0.3490
Epoch 5/5
131259/131259 [==============================] - 9s 69us/step - loss: 0.2220 - acc: 0.3506


In [18]:
from helper import nbdisplay as disp

!rm -r video
env = disp.wrap_env(gym.make("Acrobot-v1"))
observation = env.reset()
fitness = 0
  
for _ in range(steps):
  action = np.argmax(model.predict(np.array([observation]))) - 1
  observation, reward, done, _ = env.step(action)
  fitness += reward
  if done: break

env.close()
disp.show_video()
print("Video can be viewed in Google Colab, use the link on top of the page to open")
print("------ Agent: Reward {}".format(fitness))  

Video can be viewed in Google Colab, use the link on top of the page to open
------ Agent: Reward -63.0


In [0]:
import pickle
import time

filename = '04-agent-{}.model'.format(time.strftime("%Y%m%d_%H%M%S"))
file = open(filename, 'wb')
pickle.dump(model, file=file)
file.close()